<a href="https://colab.research.google.com/github/newmantic/SHAP/blob/main/SHAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import itertools
import math
from sklearn.linear_model import Ridge

class SHAPExplainer:
    def __init__(self, model, X_train):
        """
        Initialize the SHAP explainer with the model and training data.

        Parameters:
        - model: Trained machine learning model.
        - X_train: Training dataset (used for background distribution).
        """
        self.model = model
        self.X_train = X_train
        self.num_features = X_train.shape[1]

    def shap_kernel(self, S, j):
        """
        Compute the Shapley kernel weighting for a subset S and feature j.
        """
        subset_size = len(S)
        num_features = self.num_features
        return math.factorial(subset_size) * math.factorial(num_features - subset_size - 1) / math.factorial(num_features)

    def predict_with_subset(self, instance, subset):
        """
        Make a prediction using a subset of the features.

        Parameters:
        - instance: The instance to explain.
        - subset: The subset of features to include in the model.
        """
        masked_instance = np.zeros_like(instance)
        masked_instance[:, subset] = instance[:, subset]
        return self.model.predict(masked_instance)

    def shap_values(self, instance, num_samples=100):
        """
        Compute the SHAP values for a single instance.

        Parameters:
        - instance: The input instance to explain (numpy array, shape (1, num_features)).
        - num_samples: Number of subsets to sample.

        Returns:
        - shap_values: Array of SHAP values for each feature.
        """
        shap_values = np.zeros(self.num_features)

        # Iterate over all features to compute Shapley values
        for j in range(self.num_features):
            for subset_size in range(self.num_features):
                for S in itertools.combinations(range(self.num_features), subset_size):
                    if j not in S:
                        # Predictions with and without feature j
                        pred_with_j = self.predict_with_subset(instance, list(S) + [j])[0]  # extract scalar
                        pred_without_j = self.predict_with_subset(instance, list(S))[0]    # extract scalar

                        # Compute marginal contribution of feature j
                        marginal_contribution = pred_with_j - pred_without_j
                        weight = self.shap_kernel(S, j)
                        shap_values[j] += weight * marginal_contribution

        # Normalize SHAP values to sum to the model's output for the instance
        total_contribution = np.sum(shap_values)
        prediction = self.model.predict(instance)[0]
        if total_contribution != 0:
            shap_values *= prediction / total_contribution
        return shap_values

In [6]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Load the Iris dataset
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier

# Train a RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Initialize SHAP explainer for RandomForest
rf_explainer = SHAPExplainer(rf_model, X_train)

# Compute SHAP values for a specific instance
rf_shap_values = rf_explainer.shap_values(instance)

# Print SHAP values for the random forest model
print("SHAP values for the instance (RandomForest):")
for i, feature in enumerate(iris.feature_names):
    print(f"{feature}: {rf_shap_values[i]:.4f}")

SHAP values for the instance (RandomForest):
sepal length (cm): 0.2500
sepal width (cm): 0.0833
petal length (cm): 0.2500
petal width (cm): 0.4167
